# A cautionary tale of ground state energy calculation

When calculating the ground state energy of an electronic structure Hamiltonian, some care should be exercised. Converting the Hamiltonian to a sparse matrix and finding the least eigenvalue will not always be the _true_ ground state with respect to the system it represents, since the diagonalization scheme used may not inherently observe any symmetries present in the underlying physical system.

Consider the following $H_3^+$ example, consisting of 2 electrons in 6 spin-orbitals...

In [1]:
from symmer.chemistry import MoleculeBuilder
from symmer.symplectic import array_to_QuantumState
from symmer.projection import QubitTapering

basis='sto-3g'
charge=+1
geometry =[
    ("H", [0., 0.558243000, 0.]),
    ("H", [0.483452000, -0.279121000, 0.]),
    ("H", [-0.483452000, -0.279121000, 0.])
]
#geometry=[
#    ('H', (0.0,0.0,0.0)),
#    ('H', (2.45366053071732,0.0,0.0)),
#    ('H', (2.45366053071732,2.45366053071732,0.0)),
#    ('H', (0.0,2.45366053071732,0.0))
#     ]  
#charge=0
molecule = MoleculeBuilder(
    geometry=geometry, charge=charge, basis=basis, spin=0, run_fci=True, print_info=True,
    qubit_mapping_str='jordan_wigner')

Molecule geometry:
H	0.0	0.558243	0.0
H	0.483452	-0.279121	0.0
H	-0.483452	-0.279121	0.0

HF converged?   True
CCSD converged? True
FCI converged?  True

HF energy:   -1.2468600063384467
MP2 energy:  -1.2658602663569571
CCSD energy: -1.2741446169583148
FCI energy:  -1.2741444671239888


Number of qubits: 6


Naively computing the ground state energy by taking the smallest eigenvalue of the Hamiltonian does not match the FCI energy:

In [2]:
from symmer.utils import exact_gs_energy

gs_nrg, gs_vec = exact_gs_energy(molecule.H_q.to_sparse_matrix)

print(f'Least eigenvalue = {gs_nrg} | FCI error = {gs_nrg - molecule.fci_energy}')

Least eigenvalue = -1.3507306438907631 | FCI error = -0.07658617676677437


What has gone wrong here? Taking a look at the corresponding eigenvector, we see the identified state actually contains 3 particles, whereas the underlying system only contains 2:

In [3]:
psi = array_to_QuantumState(gs_vec).cleanup(zero_threshold=1e-5)
print(f'Eigenvector with eigenvalue {gs_nrg}:\n')
print(psi)

Eigenvector with eigenvalue -1.3507306438907631:

 0.062+0.102j |001011> +
 0.075+0.123j |100011> +
-0.075-0.123j |101100> +
-0.507-0.829j |111000>


In [4]:
molecule.hf_array

array([1, 1, 0, 0, 0, 0])

In [5]:
taper_hamiltonian = QubitTapering(molecule.H_q)

print(f'We are able to taper {taper_hamiltonian.n_taper} qubits from the Hamiltonian.\n')
print('The symmetry generators are\n')
print(taper_hamiltonian.symmetry_generators)
# print('\nand may be rotated onto the single-qubit Pauli operators\n')
# print(taper_hamiltonian.stabilizers.rotate_onto_single_qubit_paulis())
# print('\nvia a sequence of rotations e^{i pi/4 R} where\n')
# for index, (rot, angle) in enumerate(taper_hamiltonian.stabilizers.stabilizer_rotations):
#     print(f'R_{index} = {rot}')

We are able to taper 3 qubits from the Hamiltonian.

The symmetry generators are

 1 ZIZIIZ 
 1 IZIZIZ 
 1 IIIIZZ


In [7]:
import numpy as np

up_op, down_op = molecule.up_down_parity_operators
print(up_op, down_op)

print((-1)**(np.sum(np.bitwise_and(up_op.Z_block, molecule.hf_array))%2))
print((-1)**(np.sum(np.bitwise_and(down_op.Z_block, molecule.hf_array))%2))

S = taper_hamiltonian.symmetry_generators
print(S)
print(up_op.basis_reconstruction(S)[0][0])
print(down_op.basis_reconstruction(S)[0][0])

 1.000+0.000j ZIZIZI  1.000+0.000j IZIZIZ
-1
-1
 1 ZIZIIZ 
 1 IZIZIZ 
 1 IIIIZZ
[1 0 1]
[0 1 0]


In [8]:
from symmer.symplectic import QuantumState

print(psi.conjugate * up_op * psi)
print(psi.conjugate * down_op * psi)

hf_psi = QuantumState([molecule.hf_array])
print(hf_psi.conjugate * up_op * hf_psi)
print(hf_psi.conjugate * down_op * hf_psi)

(0.999999999999952+0j)
(-0.999999999999952+0j)
(-1+0j)
(-1+0j)


In [9]:
hf_psi

 1.000 |110000>

To counter this issue, we instead need to select the least eigenvalue that contains the _correct_ number of particles. This is implemented in `symmer.chem.exact_gs_state`

Success! We have now recovered the true ground state by enforcing that only solutions with the correct number of particles are considered. Note however it is possible that no solution is identified at first - in this case, increase the `n_eigs` parameter in `symmer.chem.exact_gs_state` to increase the search space.

In [12]:
molecule.number_operator

 3.000+0.000j IIIIII +
-0.500+0.000j ZIIIII +
-0.500+0.000j IZIIII +
-0.500+0.000j IIZIII +
-0.500+0.000j IIIZII +
-0.500+0.000j IIIIZI +
-0.500+0.000j IIIIIZ

In [14]:
H_tap = taper_hamiltonian.taper_it(molecule.hf_array)

number_op_tap = taper_hamiltonian.taper_it(molecule.hf_array, aux_operator=molecule.number_operator)
number_op_tap

 3.000+0.000j III +
-1.000+0.000j IIZ +
-0.500+0.000j IZI +
 0.500+0.000j IZZ +
-0.500+0.000j ZII +
 0.500+0.000j ZIZ

In [15]:
gs_nrg_tap, gs_vec_tap = exact_gs_energy(H_tap.to_sparse_matrix)
gs_psi_tap = array_to_QuantumState(gs_vec_tap)
gs_psi_tap

-0.991+0.000j |000> +
-0.094+0.000j |001> +
 0.000+0.000j |010> +
 0.000+0.000j |100> +
 0.094+0.000j |110>

In [16]:
gs_psi_tap.conjugate * number_op_tap * gs_psi_tap

(1.9999999999999998+0j)

In [18]:
molecule.number_operator.basis_reconstruction(S)

(array([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [1, 1, 0],
        [1, 0, 1],
        [1, 0, 0]]),
 array([0]))

In [19]:
taper_hamiltonian.stabilizers.update_sector(molecule.hf_array)
S = taper_hamiltonian.symmetry_generators
print(S)

-1 ZIZIIZ 
-1 IZIZIZ 
 1 IIIIZZ


In [20]:
from symmer.symplectic import StabilizerOp

s1 = S[0]*S[2]*S[3]
s2 = S[1]
s3 = S[2]*up_op
s4 = S[3]*up_op

S_new = s1+s2+s3+s4

S_new = StabilizerOp(S_new.symp_matrix, S_new.coeff_vec)
S_new.update_sector(ref)
print(S_new)
print()
S_new.update_sector(molecule.hf_array)
print(S_new)

AssertionError: Index out of range

In [110]:
hf_psi.conjugate * S_new[1] * hf_psi

(1+0j)

In [111]:
ref = array_to_QuantumState(gs_vec).cleanup(zero_threshold=1e-5).sort()[0].state_matrix

taper_hamiltonian.stabilizers.update_sector(ref)

print(taper_hamiltonian.symmetry_generators)

 1 ZIIZIZZI 
 1 IZIZIZIZ 
-1 IIZZIIII 
-1 IIIIZZII


 1.000+0.000j ZIZIZIZI  1.000+0.000j IZIZIZIZ
1
1
[1 0 1 1]
[0 1 0 0]


In [79]:
molecule.get_number_operator

 4.000+0.000j IIIIIIII +
-0.500+0.000j ZIIIIIII +
-0.500+0.000j IZIIIIII +
-0.500+0.000j IIZIIIII +
-0.500+0.000j IIIZIIII +
-0.500+0.000j IIIIZIII +
-0.500+0.000j IIIIIZII +
-0.500+0.000j IIIIIIZI +
-0.500+0.000j IIIIIIIZ

In [113]:
ref = array_to_QuantumState(gs_vec).cleanup(zero_threshold=1e-5).sort()[0].state_matrix

taper_hamiltonian.stabilizers = S_new
taper_hamiltonian.stabilizers.update_sector(ref)

print(taper_hamiltonian.symmetry_generators)

 1 ZIIZIZZI 
 1 IZIZIZIZ 
-1 IIZZIIII 
-1 IIIIZZII


In [75]:
H_tap = taper_hamiltonian.taper_it(molecule.hf_array)
exact_gs_energy(H_tap.to_sparse_matrix)[0] - molecule.fci_energy

0.009921175011486039

In [59]:
up_op, down_op

( 1.000+0.000j ZIZIZIZI,  1.000+0.000j IZIZIZIZ)

In [ ]:
from openfermion import FermionOperator
import numpy as np

def parity_operators(N_qubits):
    """
    note order is assumed to be spin up, spin down, spin up, spin down ... etc
    
    each op is built as product of parities of individual sites!
    
    https://arxiv.org/pdf/1008.4346.pdf
    """
    
    parity_up = FermionOperator('', 1)
    for spin_up_ind in np.arange(0,N_qubits, 2):
        parity_up*= FermionOperator('', 1) - 2*FermionOperator(f'{spin_up_ind}^ {spin_up_ind}', 1)#
    
    parity_down = FermionOperator('',1)
    for spin_down_ind in  np.arange(1,N_qubits, 2):
        parity_down*= FermionOperator('', 1) - 2*FermionOperator(f'{spin_down_ind}^ {spin_down_ind}', 1)
    
    return parity_up, parity_down

In [ ]:
from openfermion import jordan_wigner, bravyi_kitaev

up_parity, down_parity = parity_operators(len(molecule.hf_array))

In [ ]:
print(jordan_wigner(up_parity))
print()
print(jordan_wigner(down_parity))

In [ ]:
# ZIZI (up)
# IZIZ (down)
# ###


# 1 ZIIZ (a)
# 1 IZIZ (b)
# 1 IIZZ (c)


# down == b

# up == (a).(c)


In [ ]:
print(bravyi_kitaev(up_parity))
print()
print(bravyi_kitaev(down_parity))

In [ ]:
taper_hamiltonian.stabilizers.update_sector(molecule.hf_array)

print(taper_hamiltonian.symmetry_generators)

In [ ]:
# molecule.H

In [ ]:
molecule.hf_array

In [ ]:
HF_state = np.eye(2**molecule.n_qubits)[int(''.join(map(str, molecule.hf_array.tolist())), 2), :]

In [ ]:
from functools import reduce
test = [np.array([[0],[1]]) if bit==1 else np.array([[1],[0]]) for bit in molecule.hf_array]
test_state = reduce(np.kron, test)

np.allclose(test_state.reshape([-1]), HF_state.reshape([-1]))

In [ ]:
from openfermion import get_sparse_operator

HF_state.conj().T @get_sparse_operator(up_parity, n_qubits=molecule.n_qubits) @ HF_state

In [ ]:
HF_state.conj().T @get_sparse_operator(down_parity, n_qubits=molecule.n_qubits) @ HF_state

https://arxiv.org/pdf/1007.0586.pdf

parity operator for site $i$ is:

$$(-1)^{n_{i}}$$

aka -1 to the power of number of particles on site $i$

https://arxiv.org/pdf/1008.4346.pdf

In [ ]:
n_0 =  FermionOperator(f'{0}^ {0}', 1)
n_1 =  FermionOperator(f'{1}^ {1}', 1)
n_2 =  FermionOperator(f'{2}^ {2}', 1)
n_3 =  FermionOperator(f'{3}^ {3}', 1)
ni_list = [n_0, n_1, n_2, n_3]

In [ ]:
from openfermion import jordan_wigner, bravyi_kitaev

j0,j1,j2,j3 = [jordan_wigner(n) for n in ni_list]

q_list = [j0,j1,j2,j3]

In [ ]:
j0

In [ ]:
from scipy.linalg import expm

ttt = [1j*np.pi*get_sparse_operator(op, n_qubits=4).todense() for op in q_list]

pol = reduce(lambda x,y: x+y, ttt)

exp_pol = expm(pol)
np.where(expm(exp_pol)!=0)

np.diag(exp_pol)

In [ ]:
H_mat = get_sparse_operator(molecule.H).todense()

In [ ]:
H_mat@exp_pol == exp_pol@H_mat

In [ ]:
FermionOperator()

In [ ]:
import os
import yaml

In [ ]:
molecule.hf_array.tolist()


In [ ]:
data = {}
H_dict = molecule.H_q.to_dictionary

for key in H_dict.keys():
    H_dict[key] = float(H_dict[key].real)

# H_dict = {'cat': H_dict['IIII']}
data['H'] = H_dict
data['taper_reference'] = molecule.hf_array.tolist()

In [ ]:
data

In [ ]:
out = os.path.join(os.getcwd(), 'H2_JW.yaml')
with open(out, 'w') as file:
    yaml.dump(data, file)

In [ ]:
with open(out, "r") as stream:
    try:
        test = yaml.full_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [ ]:
test

In [ ]:
from symmer.chemistry import MoleculeBuilder
from symmer.symplectic import array_to_QuantumState

basis='sto-3g'
charge=0
geometry =[
    ("H", [0., 0., 1.]),
    ("H", [0., 1., 0]),
    ("H", [0., 1., 1.]),
    ("H", [0., 0., 0.])
]

# geometry =[
#     ("H", [0., 0., 0.]),
#     ("H", [0., 0., 0.74]),
# ]
molecule = MoleculeBuilder(geometry=geometry, charge=charge, basis=basis, spin=0, run_fci=True, print_info=True)